In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
sine_a = np.sin(np.linspace(0,10*2*np.pi, 64,endpoint=False))
plt.plot(sine_a)

rng = RandomState(20190329)
amps = rng.randn(300)
signals = amps[:,None] * sine_a[None]
plt.figure(figsize=(12,3))
plt.plot(signals.T, color=seaborn.color_palette()[0], lw=0.25);

In [ ]:
A = []
for i_component in range(1,17):
    i_freq = i_component // 2
    if (i_component % 2 == 0) | (i_freq == 0):
        a = np.cos(np.linspace(0, i_freq*2*np.pi,16, endpoint=False))
    else:
        a = np.sin(np.linspace(0, i_freq*2*np.pi,16, endpoint=False))
    A.append(a)
A = np.array(A)

In [ ]:
A = A / np.sqrt(np.sum(np.square(A), axis=1, keepdims=True))

In [ ]:
w = [1.2,-3,2,3,2,1,0,-1,-2,0,1,2,1,0,1,0.1]

In [ ]:
np.fft.rfft(w) / np.sqrt(16)

In [ ]:
np.mean(np.abs(np.linalg.inv(A).T - A))

In [ ]:
plt.plot(A.T)

In [ ]:
np.linalg.inv(A).T[0]

In [ ]:
plt.plot(np.linalg.inv(A))

In [ ]:
np.dot(A,w)

In [ ]:
A

In [ ]:
sine_a = np.sin(np.linspace(0,5*2*np.pi, 32,endpoint=False))
plt.plot(sine_a)
for i in range(0,33,8):
    plt.axvline(x=i, color='black')

In [ ]:
plt.figure(figsize=(16,3))
plt.plot(np.abs(np.fft.rfft(np.sin(np.linspace(0,4.5*2*np.pi, 100, endpoint=False,)))))
plt.plot(np.abs(np.fft.rfft(np.sin(np.linspace(0,5*2*np.pi, 100, endpoint=False,)))))

plt.xticks(range(0,51,2));